In [1]:
import pandas as pd
import openpyxl, pprint, os, sys
from datetime import datetime, timedelta
from openpyxl.styles import Alignment

In [2]:
# csvファイルを読み込む
products = pd.read_csv("../input/products - copy.csv", encoding="shift_jis")
orders = pd.read_csv("../input/orders - copy.csv", encoding="shift_jis")

In [3]:
# テンプレートを読み込む
quotation_wb = openpyxl.load_workbook("../template/quotation_template.xlsx")

In [16]:
# 対象の注文IDを対話形式で入力
target_id = input("注文IDを入力して下さい")

注文IDを入力して下さいO0002


In [17]:
# 情報を元に見積書を作成
# 対象の取引データを抽出
target_orders = orders[orders["order_id"] == target_id]
target_orders

order_id customer_name  order_date product_id  quantity
3    O0002           鈴木健  2025-05-11       P010         1
4    O0002           鈴木健  2025-05-11       P008         5
5    O0002           鈴木健  2025-05-11       P009         4

In [7]:
products

product_id  product_name  unit_price unit               note tax_type  \
0       P001       ノートパソコン      100000    台  15.6インチ SSD 512GB       課税   
1       P002      レーザープリンタ       40000    台          両面印刷対応 A4       課税   
2       P003     インクカートリッジ        3000    個         ブラック・カラー対応       課税   
3       P004      HDMIケーブル        1200    本        1.5m 金メッキ仕様       課税   
4       P005   USBメモリ 64GB        2000    本          USB 3.0対応       課税   
5       P006         無線マウス        2500    個       2.4GHz ワイヤレス       課税   
6       P007  液晶モニター 24インチ       18000    台     IPS 非光沢 HDMI対応       課税   
7       P008  キーボード（日本語配列）        3500    個           USB接続タイプ       課税   
8       P009   Webカメラ フルHD        5000    個      1080p対応 マイク内蔵       課税   
9       P010     有線LANアダプタ        2800    個      USB-C対応 ギガビット       課税   

  category         supplier  
0     PC本体   TechSupplier A  
1     周辺機器  OfficeGear Inc.  
2      消耗品         InkWorks  
3     周辺機器        CableMart  
4     周辺機器      MemoryWorld  
5     周辺機器   Peripheral Co.  
6     周辺機器     Display Tech  
7     周辺機器   Keyboards Ltd.  
8     周辺機器      VisionWorks  
9     周辺機器       NetConnect

In [22]:
# データを結合
merged = target_orders.merge(products, on="product_id", how="left")
merged

order_id customer_name  order_date product_id  quantity  product_name  \
0    O0002           鈴木健  2025-05-11       P010         1     有線LANアダプタ   
1    O0002           鈴木健  2025-05-11       P008         5  キーボード（日本語配列）   
2    O0002           鈴木健  2025-05-11       P009         4   Webカメラ フルHD   

   unit_price unit           note tax_type category        supplier  
0        2800    個  USB-C対応 ギガビット       課税     周辺機器      NetConnect  
1        3500    個       USB接続タイプ       課税     周辺機器  Keyboards Ltd.  
2        5000    個  1080p対応 マイク内蔵       課税     周辺機器     VisionWorks

In [19]:
# 金額列の章啓
merged["subtotal"]  = merged["unit_price"] * merged["quantity"]

In [20]:
merged

order_id customer_name  order_date product_id  quantity  product_name  \
0    O0002           鈴木健  2025-05-11       P010         1     有線LANアダプタ   
1    O0002           鈴木健  2025-05-11       P008         5  キーボード（日本語配列）   
2    O0002           鈴木健  2025-05-11       P009         4   Webカメラ フルHD   

   unit_price unit           note tax_type category        supplier  subtotal  
0        2800    個  USB-C対応 ギガビット       課税     周辺機器      NetConnect      2800  
1        3500    個       USB接続タイプ       課税     周辺機器  Keyboards Ltd.     17500  
2        5000    個  1080p対応 マイク内蔵       課税     周辺機器     VisionWorks     20000

In [21]:
# 見積書への入力作業
# 明細の開始行
start_row = 19

# 右詰めスタイルを作成
right_align = Alignment(horizontal="right")

# 日付設定
today = datetime.today()
due_date = today + timedelta(days=30)

# 明細行描写
for i, (_, row) in enumerate(merged.iterrows()):
    quotation_ws.cell(row=start_row + i, column=2, value=i + 1) # No(連番)
    quotation_ws.cell(row=start_row + i, column=3, value=row["product_name"]) # 商品名
    quotation_ws.cell(row=start_row + i, column=4, value=row["quantity"]) # 数量
    quotation_ws.cell(row=start_row + i, column=5, value=row["unit_price"]) # 単価(円)
    quotation_ws.cell(row=start_row + i, column=6, value=row["subtotal"]) # 金額(円)
    
# 合計金額
quotation_ws["F35"] = merged["subtotal"].sum()

# 宛名
quotation_ws["B3"] = merged.iloc[0]["customer_name"] + "  様"

# 見積日
cell = quotation_ws["F12"]
cell.value = today.strftime("%Y年%m月%d日")
cell.alignment = right_align

# 振込期限
cell = quotation_ws["F13"]
cell.value = due_date.strftime("%Y年%m月%d日")
cell.alignment = right_align
    
quotation_wb.save(f"../output/quotation{target_id}.xlsx")